# Macros 

## Recap

Last time we spoke about how syntax is represented in Julia. 

Recall, Julia syntax is represented as an `Expr` object.

In [1]:
dump(:(1+2))

Expr
  head: Symbol call
  args: Array{Any}(

(3

,))
    1: Symbol +
    2: Int64 1
    3: Int64 

2


Working with `Expr`s directly is a bit yucky, so we can use *quoting* instead.

In [2]:
ex = quote 
    1+2
end

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:2 =#
    1 + 2
end

We evaluate expressions with 

In [3]:
eval(ex)

3

This executes in the *global* scope

In [4]:
c=0
ex = :(c+=1)
function incerement_10()
    for n in 1:10
        eval(ex)
    end
    return nothing
end
incerement_10()
c

10

Wierd! (^this is a very bad idea btw)

### why?
There are advantages to representing syntax internally (as opposed to as a string) -- introsepection, more conveninent destructuring of code. For most use cases, it doesn't matter. 

### Uses
Typically, we can achieve all we need with normal julia code, but metaprogramming can come in handy, mainly for automatic code generation -- I've used it for testing or generalising methods for differenet types. E.g.

In [5]:
struct my_angle
    a::Float64
end

for f in (:sin, :cos, :tan, :tanh, :cosh, :sinh, :exp)
    @eval Base.$f(x::my_angle) = my_angle($f(x.a))
end
a = my_angle(1.0)
sin(a)

my_angle(0.8414709848078965)

# Macros
Are another big use of metaprogramming.

By being able to represent code within Julia we can create `function`s to manipulate `Expr`essions and then `eval`uate them at run-time. 
```
f(expr1)->eval(expr2)
```
**So, what's the point in Macros?**

Over and above `f(expr1)->eval(expr2)`, macros allow code to be executed *before* the full program is run (before run time).

Macros allow us to manipulate and insert code at *compile time* (parse time?). This is in constrast to `eval(expr)` which only evaluate at run time. 

I think this is easiest to demonstrate by examples. 

First, define a macro to do some work

In [6]:
macro mymacro(x)
    @show x
    y=1+x
    @show y
    return :($(esc(x))+$y)
end

@mymacro (macro with 1 method)

Now define an "equivalent" expression

In [7]:
expr(x) = quote 
    @show $x
    y = 1+$x
    @show y
    $x+y
end

expr (generic function with 1 method)

To see the difference between evaluating an expression and a macro, consider the following two functions. 

In [8]:
f(z) = z + @mymacro(1)

x = 1
y = 2


f (generic function with 1 method)

In [9]:
g(z) = z + eval(expr(1))

g (generic function with 1 method)

For a third comparison, also define a third equivalent function without macros or expressions

In [10]:
function h(z)
    x=1
    @show x
    y = 1+x
    @show y
    return z + x+y
end

h (generic function with 1 method)

Already, the keen eyed viewer might have noticed the difference. 

Now observe what happens when we execute the functions. 

In [11]:
f(1)

4

In [12]:
g(1)

1 = 1
y = 2


4

In [13]:
h(1)

x = 1
y = 2


4

Another (more concrete) way to spot the difference is to look at the lowered code. The lowered version of the code is the code which is actually run when we execute the function. 

```julia 
@code_lowered f(1)
CodeInfo(
1 ─ %1 = 1 + 2
│   %2 = z + %1
└──      return %2
)
```
---
```julia 
@code_lowered g(1) 
CodeInfo(
1 ─ %1 = Main.expr(1)
│   %2 = Main.eval(%1)
│   %3 = z + %2
└──      return %3
)
```


---
```julia 
@code_lowered h(1)
CodeInfo(
1 ─       x = 1
│         value@_4 = x
│   %3  = x
│   %4  = Base.repr(%3)
│         Base.println("x = ", %4)
│         value@_4
│         y = 1 + x
│         value@_3 = y
│   %9  = y
│   %10 = Base.repr(%9)
│         Base.println("y = ", %10)
│         value@_3
│   %13 = z + x + y
└──       return %13
)
```

Notice that `f(1)` contains only the expression which results from the macro (line %2); `g(1)` contains a call to `eval(expr(1))` (lines 1% and 2%); `h(1)` is like `g(1)` except the expression is directly implmented in the function call. 

Another example is the following: 

In [14]:
# https://nbviewer.org/github/crstnbr/JuliaWorkshop18/blob/master/2%20Tips%20and%20tricks/optional_field_inheritance.ipynb
macro add_generic_fields()
    return esc(quote
        one_field::Int
        another::Int
        yet_another::Float64
        a::Vector
        b::Vector
        f::Function
    end)
end

@add_generic_fields (macro with 1 method)

In [15]:
struct MyType
    @add_generic_fields
    specific_field1::Int
    # more fields # 
end
struct YourType
    @add_generic_fields
    specific_field2::Int
    # more fields # 
end

In [16]:
fieldnames(MyType)

(:one_field, :another, :yet_another, :a, :b, :f, :specific_field1)

In [17]:
fieldnames(YourType)

(:one_field, :another, :yet_another, :a, :b, :f, :specific_field2)

I don't think there is a way to do this without macros. Unlike function `struct` definitions are never evaluated, so there is no possibility to insert an expression. i.e. you can't do 

In [18]:
# struct MyType 
#     eval(generic_fields_expr)
#     specific_field
# end

The closest I can get is; 

In [19]:
function struct_with_fields(name::Symbol,specific_field::Symbol)
    return eval(quote 
        struct $name
            # generic fields
            one_field::Int
            another::Int
            yet_another::Float64
            a::Vector
            b::Vector
            f::Function
            $specific_field
        end
    end)
end

struct_with_fields (generic function with 1 method)

In [20]:
struct_with_fields(:HerType,:her_field)
fieldnames(HerType)

(:one_field, :another, :yet_another, :a, :b, :f, :her_field)

One place where macros are useful is constructing big floats. 

Consider writing a long float in an expression

In [21]:
a=0.123456789012345678901234567890-0.123456789012345678

0.0

In [22]:
a==0.0

true

The problem is, when the long float is parsed into julia code, it is parsed as a regular Float64, thereby dropping the last 12 digits. A macro can be used to manipulate the long float at parse time, into a `BigFloat`

In [23]:
a=big"0.123456789012345678901234567890"-0.123456789012345678

1.531348335790184029713273048400878906249999999999999999999999582792173561026788e-18

Since the number is parsed as a float, we can't even do this

In [24]:
BigFloat(0.123456789012345678901234567890)

0.12345678901234567736988623209981597028672695159912109375

The code above returs a BigFloat, but is rounded. C.f.

In [25]:
big"0.123456789012345678901234567890"

0.1234567890123456789012345678899999999999999999999999999999999999999999999999996

In [26]:
big"0.123456789012345678901234567890"-BigFloat(0.123456789012345678901234567890)

1.531348335790184029713273048400878906249999999999999999999999582792173561026788e-18

### Comments 
Macros are rarely necessary. Typically things are better done with a regular old function. And functions are much easier to write. 

Nonetheless, understanding them helps us understand how code is processes, and also when a macro might be handy. 

The main uses of macros I have come across are 
1) code generation (loop unrolling, see Unrolled.jl)
2) domain specific language/simplifying syntax (see JuMP.jl, GLM.jl, SparseArrays.jl)
3) access to the code and value at the same time (i.e. `@show` -- probably my favourite macro, great for debugging)

but typically macros are not *essential* for these. 

# How to write a macro 
(https://jkrumbiegel.com/pages/2021-06-07-macros-for-beginners/)

A macro is a lot like a function `f(expr1) -> expr2`, and `expr2` is pasted into the code where the macro is called. However, there are two main conceptual differences. 

The first big conceptual difference to note when writing a macro is that it is executed before the code is run, so it does not have access to any run-time values. 

A covenient tool for determining what infomation you have at parse time is `Meta.@dump x` (equivalent to `dump(:(x))`)

In [27]:
Meta.@dump x

Symbol x


So all the macro knows, is that `x` is a `Symbol`. At this point, the macro knows no information about the value of `x`. 

Let's make a simple macro which will help us see what the macro knows

In [28]:
macro show_the_insides(x)
    @show x 
    @show typeof(x)
end 

@show_the_insides (macro with 1 method)

In [29]:
x = 1 
@show_the_insides x;

x = :x
typeof(x) = Symbol


In [30]:
@show_the_insides 1;

x = 1
typeof(x) = Int64


In [54]:
@show_the_insides 1+2;

x = :(1 + 2)
typeof(x) = Expr


Compare this to the run-time information 

In [32]:
dump(x)

Int64 1


In [33]:
dump(1+2)

Int64 3


In contrast, the run-time knows nothing about the variable name or expression, but does know its value. 

The value of variables passed to macros can be accessed at run-time by interpolating the value into the returned expression. 


In [34]:
macro show_the_insides_and_value_when_evaled(x)
    @show x 
    @show typeof(x)
    return :(@show $x)
end 

@show_the_insides_and_value_when_evaled(x);

x = :x
typeof(x) = Symbol
x = 1


The second big conceptual difference is to do with *hygiene*. 

Recall that a macro takes an expression,manipulates it, and pastes the resulting expression where the macro was placed (when we run a macro in the REPL the resulting expression is immediately evaluated, so we don't see it.)

Now, imagine the returned expression from the macro contains a variable name which the user has defined, but was not passed to the macro. The macro could end up having side effects. 

For example, in an unhygienic world, we would have something like this.

```julia 
macro a_macro(x)
    return quote 
        important_user_information = nothing 
       ($x)^2
    end
end    
```

Now, suppose `a_macro` is used in the following context
```julia 
important_user_information = simulation_which_takes_hours(seed=1)
y = 2
@a_macro(y)
```

Conceptually, the resulting code would be 
```julia 
important_user_information = simulation_which_takes_hours(seed=1)
y = 2
important_user_information = nothing 
y^2
```

Oh no!

Thankfully, this is not what actually happens. Instead, we have hygiene rules which make it hard (not impossible) to do things like this. 

So, let's start with a simple example. The following macro prints out a value and variable name (much like `@show`)

In [35]:
macro show_value(currency)
    @show currency
    str = string(currency)
    return quote 
        println("you have ",$currency, " ",$str)
    end
end

@show_value (macro with 1 method)

In [36]:
dollars = 20
@show_value(dollars)

currency = :dollars
you have 20 dollars


The syntax is much like a function. 

Things to note:
1) the macro returns a `Expr`ession
2) the macro has access to the variable name (unlike a function)

I used `@show` within the macro to remind us that the macro only has access to the variable name (as a symbol), in this case `currency = :dollars`.

Before the returned expression, we evaluated `str = string(currency)`. 

When we evaluate the macro, currency takes the value `:dollars` so `str = string(:dollars)` which evaluates to `"dollars"`.

In the returned expression the `$currency` refers to the value at run-time; `$str` interpolates the string `"dollars"` in its place. 

One of the best ways to work out what is actually going on within your macro is to use the `@macroexpand` macro, which displays the returned expression 


In [37]:
@macroexpand @show_value(dollars)

currency = :dollars


quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:5 =#
    Main.println("you have ", Main.dollars, " ", "dollars")
end

So, now we are hopefully starting to understand how macros behave. Let's start to look at hygiene. 

First, what does a macro do when the returned expression introduces new variables. 

In [38]:
macro show_relative_value(currency)
    # @show currency
    str = string(currency)
    return quote 
        half = $currency/2
        double = $currency*2
        println("you have ",$currency, " ",$str)
        println("which is more than ", half, " ", $str)
        println("but less than ", double, " ", $str)
    end
end

@show_relative_value (macro with 1 method)

Recall that the macro pastes the returned expression in place where it is evaluated. 

Evaluating this macro gives

In [39]:
@show_relative_value(dollars)

you have 20 dollars
which is more than 10.0 dollars
but less than 40 dollars


The returned expression which is pasted in the macros place is

In [40]:
@macroexpand @show_relative_value(dollars)

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:5 =#
    var"#20#half" = Main.dollars / 2
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:6 =#
    var"#21#double" = Main.dollars * 2
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:7 =#
    Main.println("you have ", Main.dollars, " ", "dollars")
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:8 =#
    Main.println("which is more than ", var"#20#half", " ", "dollars")
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:9 =#
    Main.println("but less than ", var"#21#double", " ", "dollars")
end

Notice that the variables the macro introduces have weird names. Julia has automaticall generated unique names which (almost surely) won't clash with any others. It does this via `gensym()`.

The variable `Main.dollars` also appears in the returned expression, since we interpolated its value in. 

By default, macros always look for interpolated values within their current module (in this case `Main`).

To demonstrate, let's put out macro in a module 

In [41]:
module M
    export @show_value_within_M
    macro show_value_within_M(currency)
        str = string(currency)
        return quote 
            println("you have ",$currency, " ",$str)
        end
    end
end
using .M 

In [42]:
#@show_value_within_M(dollars)
# throws error 

```julia 
julia> @show_value_within_M(dollars)
ERROR: UndefVarError: dollars not defined
Stacktrace:
 [1] top-level scope
   @ REPL[24]:6
```

In [43]:
@macroexpand @show_value_within_M(dollars)

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:6 =#
    Main.M.println("you have ", Main.M.dollars, " ", "dollars")
end

By default, julia has looked for the `dollars` variable within the module. But we want to acccess the variable within the current context (`Main`).

We do this by escaping 

In [44]:
module M2
    export @show_value_within_M2
    macro show_value_within_M2(currency)
        str = string(currency)
        return quote 
            println("you have ",$(esc(currency)), " ",$str)
        end
    end
end
using .M2

In [45]:
@show_value_within_M2(dollars)

you have 20 dollars


In [46]:
@macroexpand @show_value_within_M2(dollars)

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:6 =#
    Main.M2.println("you have ", dollars, " ", "dollars")
end

Now the macro just calls `dollars` which is within whatever context the macro is evaluated. 

You should only escape variables which the macros knows about (those which have been explicitly passed to the macro). Otherwise you might overwrite the users variables. 

An example of bad escaping. 

In [47]:
half = 1/2

macro show_relative_value(currency)
    # @show currency
    str = string(currency)
    expr = quote 
        half = $currency/2
        double = $currency*2
        println("you have ",$currency, " ",$str)
        println("which is more than ", half, " ", $str)
        println("but less than ", double, " ", $str)
    end
    return esc(expr)
end

@show_relative_value (macro with 1 method)

In [48]:
@show_relative_value(dollars)

you have 20 dollars
which is more than 10.0 dollars
but less than 40 dollars


In [49]:
@macroexpand @show_relative_value(dollars)

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:7 =#
    half = dollars / 2
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:8 =#
    double = dollars * 2
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:9 =#
    println("you have ", dollars, " ", "dollars")
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:10 =#
    println("which is more than ", half, " ", "dollars")
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/macros.ipynb:11 =#
    println("but less than ", double, " ", "dollars")
end

In [50]:
half

10.0

## Multiple dispatch

Macros can use multiple dispatch, but remember, they only have compile time information. 

For example 

In [51]:
macro foo(x::Int)
    return 2*x
end 
macro foo(x::Symbol)
    return :($(string(x)))
end

@foo (macro with 2 methods)

In [52]:
@foo(2) # dispatches to foo(::Int)

4

In [53]:
x=2
@foo(x) # displates to foo(::Symbol)

"x"

See also: 

this is great https://jkrumbiegel.com/pages/2021-06-07-macros-for-beginners/

this is pretty cool https://github.com/FugroRoames/RoamesNotebooks/blob/master/A%20practical%20introduction%20to%20metaprogramming%20in%20Julia.ipynb 

https://stackoverflow.com/questions/58137512/why-use-macros-in-julia

https://github.com/cstjean/Unrolled.jl

https://nbviewer.org/github/crstnbr/JuliaWorkshop18/blob/master/2%20Tips%20and%20tricks/optional_field_inheritance.ipynb

https://docs.julialang.org/en/v1/manual/metaprogramming/#man-macros 

The last comment on this post https://stackoverflow.com/questions/39627029/julia-how-does-inline-work-when-to-use-function-vs-macro/39627556